In [1]:
import numpy as np
import scipy
import sklearn
import matplotlib.pyplot as plt
import matplotlib as mpl
import sympy
import pandas
import torch
# import jtplot module in notebook
from jupyterthemes import jtplot

# choose which theme to inherit plotting style from
# onedork | grade3 | oceans16 | chesterish | monokai | solarizedl | solarizedd
jtplot.style(theme='onedork')

In [4]:
# 미분법 예제
# 0. 직접 미분

x = sympy.Symbol('x')
f = (x**2 + 2*x)*sympy.log(x)
df = sympy.diff(f, x)
df

(2*x + 2)*log(x) + (x**2 + 2*x)/x

In [3]:
sympy.simplify(df)

x + 2*(x + 1)*log(x) + 2

In [3]:
#1. 수치 미분

def numer_deriv(f, x, h=0.001, method="center"):
    """
    f : 미분합 함수로 주어진 위치에서 함숫값 계산을 위해 사용
    x : 미분계수를 구할 변수의 위치로,
        일변수인 경우 Int, float
        다변수인 경우 numpy array, vector
    h : 비율을 구할 작은 구간
    """
    if type(x) in (float, int):
        grad = [0,0]
        x_ = [x]
        var_type = 'scalar'
    else :
        grad = np.zeros(x.shape)
        x_ = x.copy().astype('float32')
        var_type = 'vector'
    
    for i, xi in enumerate(x_):
        original_value = x_[i]
        
        if method=='forward':
            x_[i] = original_value + h
        else:
            x_[i] = original_value + (h/2)
        
        if var_type == 'scalar':
            gradplus = f(x_[i])
        else :
            gradplus = f(x_)
        
        if method=='forward':
            x_[i] = original_value
        else :
            x_[i] = original_value - (h/2)
        
        if var_type =='scalar':
            gradminus = f(x_[i])
        else :
            gradminus = f(x_)
        
        grad[i] = (gradplus - gradminus) / h
        
    if var_type == 'scalar':
        return grad[0]
    else :
        return grad

In [4]:
f = lambda x : (x**2 + 2*x)*np.log(x)
df = lambda x : 2*(x+1)*np.log(x) + (x+2)

numer_deriv(f,1)

2.999999999999666

In [5]:
print(numer_deriv(f,1,h=0.5, method="forward"))

4.257383635135726


In [6]:
print(numer_deriv(f,1,h=0.5, method="center"))

2.9997299032915508


In [5]:
f_xy = lambda x : (x[0]**2 + 2*x[0])*np.log(x[1])
numer_deriv(f_xy, np.array([1, 2]))

array([2.77255299, 1.49889143])

In [6]:
x = sympy.Symbol('x')
y = sympy.Symbol('y')
f_xy_sympy = (x**2 + x*2)*sympy.log(y)
df_xy_x = sympy.diff(f_xy_sympy, x)
df_xy_y = sympy.diff(f_xy_sympy, y)

print(df_xy_x)
print(df_xy_y)
print(f"{df_xy_x.evalf(subs={x:1.0, y:2.0})}")
print(f"{df_xy_y.evalf(subs={x:1.0, y:2.0})}")

(2*x + 2)*log(y)
(x**2 + 2*x)/y
2.77258872223978
1.50000000000000


In [7]:
df_xy_x

(2*x + 2)*log(y)

In [8]:
# 자동미분

np.random.seed(0)

x = np.random.rand(6).reshape(2,3)

x_tensor = torch.tensor(x)
x_from_numpy = torch.from_numpy(x)
x_Tensor = torch.Tensor(x)
x_as_tensor = torch.as_tensor(x)

print(x, x.dtype)
print(x_tensor, x_tensor.dtype, x_tensor.requires_grad)
print(x_from_numpy, x_from_numpy.dtype, x_from_numpy.requires_grad)
print(x_Tensor, x_Tensor.dtype, x_Tensor.requires_grad) # float 32bits 
print(x_as_tensor, x_as_tensor.dtype, x_as_tensor.requires_grad)

[[0.5488135  0.71518937 0.60276338]
 [0.54488318 0.4236548  0.64589411]] float64
tensor([[0.5488, 0.7152, 0.6028],
        [0.5449, 0.4237, 0.6459]], dtype=torch.float64) torch.float64 False
tensor([[0.5488, 0.7152, 0.6028],
        [0.5449, 0.4237, 0.6459]], dtype=torch.float64) torch.float64 False
tensor([[0.5488, 0.7152, 0.6028],
        [0.5449, 0.4237, 0.6459]]) torch.float32 False
tensor([[0.5488, 0.7152, 0.6028],
        [0.5449, 0.4237, 0.6459]], dtype=torch.float64) torch.float64 False


In [9]:
x[0,0] = 100
print(x,x.dtype)

[[100.           0.71518937   0.60276338]
 [  0.54488318   0.4236548    0.64589411]] float64


In [10]:
print(x_tensor,x_tensor.dtype, x_tensor.requires_grad)

print(x_from_numpy, x_from_numpy.dtype, x_from_numpy.requires_grad)

tensor([[0.5488, 0.7152, 0.6028],
        [0.5449, 0.4237, 0.6459]], dtype=torch.float64) torch.float64 False
tensor([[100.0000,   0.7152,   0.6028],
        [  0.5449,   0.4237,   0.6459]], dtype=torch.float64) torch.float64 False


In [11]:
x_tensor_grad = torch.tensor(x, requires_grad=True)
print(x_tensor_grad, x_tensor_grad.dtype, x_tensor_grad.requires_grad)

tensor([[100.0000,   0.7152,   0.6028],
        [  0.5449,   0.4237,   0.6459]], dtype=torch.float64,
       requires_grad=True) torch.float64 True


In [12]:
x = torch.tensor([1.0], requires_grad=True)
f = (x**2 + 2*x) * torch.log(x)
print(x)
print(f)
print(x.grad)

tensor([1.], requires_grad=True)
tensor([0.], grad_fn=<MulBackward0>)
None


In [15]:
torch.autograd.backward(f, retain_graph=True) # memory에 남겨두는 역할
print(x.grad)

tensor([6.])


In [16]:
df = torch.autograd.grad(f, x, retain_graph=True)
print(df)

(tensor([3.]),)


In [17]:
df = torch.autograd.grad(f,(x,x), retain_graph=True)
print(df)

(tensor([3.]), tensor([3.]))


In [18]:
x = torch.tensor([1.0], requires_grad=True)
y = torch.tensor([2.0], requires_grad=True)
f_xy = (x**2 + 2*x) * torch.log(y)

torch.autograd.backward(f_xy, retain_graph=True)
print(x.grad)

tensor([2.7726])


In [19]:
df = torch.autograd.grad(f_xy, (x,y), retain_graph=True)
print(df)

(tensor([2.7726]), tensor([1.5000]))


In [6]:
# 자동미분 구현

def times(x,y):
    return x*y, (x, y)

def times_deriv(cache, dout=1):
    return cache[1]*dout, cache[0]*dout

TIMES = {
    'f' : times,
    'df' : times_deriv
}

v, cache = TIMES['f'](2,3) # 함수 표현 방식 익혀두기.
dx, dy = TIMES['df'](cache)

print("dx={}, dy={}".format(dx,dy))

dx=3, dy=2


In [7]:
def add(x, y):
    return x+y, (x,y)

def add_deriv(cache, dout=1):
    return dout, dout

ADD = {
    'f' : add,
    'df' : add_deriv
}

def log(x):
    return np.log(x), x

def log_deriv(cache, dout=1):
    return (1/cache)*dout

LOG = {
    'f' : log,
    'df' : log_deriv
}

In [10]:
x = 1.; y = 2.

a, cache_a = TIMES['f'](x,x)
b, cache_b = TIMES['f'](2,x)
c, cache_c = ADD['f'](a,b)
d, cache_d = LOG['f'](y)
z, cache_z = TIMES['f'](c, d)

dx = dy = 0.

dc, dd = TIMES['df'](cache_z, 1)
dy  = LOG['df'](cache_d, dd)
da, db = ADD['df'](cache_c, dc)
_, dx_ = TIMES['df'](cache_b, db); dx += dx_;
dx_, dx__ = TIMES['df'](cache_a, da); dx += dx_ + dx__;

print(f'backward pass dx = {dx}, dy = {dy}')

backward pass dx = 2.772588722239781, dy = 1.5


In [13]:
x = torch.tensor([1.], requires_grad=True)
y = torch.tensor([2.], requires_grad=True)
z = (x**2 + x*2)*torch.log(y)

# grad_outputs : upstream derivative
dz = torch.autograd.grad(z, (x,y), grad_outputs=torch.tensor([2.]), retain_graph=True)
print(dz) # upstream derivative가 2이기 때문에 기존 derivative의 2배가 결과로 나왔다

(tensor([5.5452]), tensor([3.]))
